In [ ]:
import pandas as pd
import sys
import numpy as np
sys.path.append("../helper_functions")
import defillama_utils as dfl
import duneapi_utils as du
sys.path.pop()

In [ ]:
chains = [
    #L2s
     ['Optimism', 'L2']
    ,['Base', 'L2']
    ,['Arbitrum', 'L2']
    #non-dune L2s
    ,['ZkSync Era', 'L2']
    ,['Polygon zkEVM', 'L2']
    ,['Starknet', 'L2']
    ,['Linea', 'L2']
    ,['Mantle', 'L2']
    ,['Scroll', 'L2']
    ,['Boba', 'L2']
    ,['Metis', 'L2']
    ,['opBNB', 'L2']
    ,['Rollux', 'L2']
    ,['Manta', 'L2']
    ,['Kroma','L2']
    #L1
    ,['Ethereum', 'L1']
    #Others
    ,['Fantom', 'L1']
    ,['Avalanche', 'L1']
    ,['Gnosis' , 'L1']
    ,['Celo', 'L1']
    ,['Polygon', 'L1']
    ,['BSC', 'L1']
]

protocols = [
        ['aevo', 'L2',['optimism','arbitrum','ethereum']]
    ]

In [ ]:
p_agg = []
for p in protocols:
        d = dfl.get_single_tvl(p[0],p[2])
        d['chain_prot'] = p[0]
        d['layer'] = p[1]
        # d['prot_chain'] = c
        p_agg.append(d)
df = pd.concat(p_agg)

df_sum = df.groupby(['date','chain_prot','layer']).sum(['usd_value'])
df_sum.reset_index(inplace=True)

df_sum = df_sum[['date','usd_value','chain_prot','layer']]
df_sum = df_sum.rename(columns={'chain_prot':'chain','usd_value':'tvl'})

print(df_sum.sample(5))

In [ ]:
c_agg = []
for c in chains:
        d = dfl.get_historical_chain_tvl(c[0])
        d['chain'] = c[0]
        d['layer'] = c[1]
        c_agg.append(d)

df_ch = pd.concat(c_agg)
df = pd.concat([df_ch,df_sum])
# Rename
df.loc[df['chain'] == 'Optimism', 'chain'] = 'OP Mainnet'
df.loc[df['chain'] == 'BSC', 'chain'] = 'BNB'
df.loc[df['chain'] == 'aevo', 'chain'] = 'Aevo'

df['date'] = pd.to_datetime(df['date'])
print(df.sample(5))

print(df['chain'].unique())

In [ ]:
# Write to Dune
du.write_dune_api_from_pandas(df, 'dfl_chain_tvl',\
                             'TVL for select chains from DefiLlama')